In [10]:
import os
from dotenv import load_dotenv
from llama_index.core import (
    VectorStoreIndex, 
)
from llama_index.core.agent.workflow import AgentStream, ToolCallResult
load_dotenv()

True

In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
from llama_index.core.agent.workflow import ReActAgent
from llama_index.core.workflow import Context
from llama_index.llms.google_genai import GoogleGenAI

llm_gemini = GoogleGenAI(model="gemini-2.0-flash", api_key=os.environ["GEMINI_API_KEY"])

In [13]:
import pandas as pd

# dataset con diferentes clientes, su email y el detalle de una compra
df_shoppings = pd.DataFrame({
    "Cliente": ["Juan Pérez", "Ana Gómez", "Luis Martínez", "Ana Gómez", "Carlos López"],
    "Email": ["juan.perez@email.com", "ana.gomez@email.com", "luis.martinez@email.com", "ana.gomez@email.com", "carlos.lopez@email.com"],
    "DetalleCompra": [
        "Compra de laptop - Modelo: Dell XPS 13, Procesador: Intel i7, RAM: 16GB, Almacenamiento: 512GB SSD",
        "Compra de smartphone - Modelo: iPhone 14 Pro, Almacenamiento: 256GB, Color: Negro",
        "Compra de tablet - Modelo: Samsung Galaxy Tab S8, Pantalla: 11 pulgadas, Almacenamiento: 128GB",
        "Compra de audífonos - Modelo: Sony WH-1000XM5, Tipo: Over-ear, Cancelación de ruido activa",
        "Compra de monitor - Modelo: LG UltraGear 27GP850, Tamaño: 27 pulgadas, Resolución: QHD, Frecuencia: 165Hz"
    ]
})

# dataset con diferentes clientes, su email y el detalle de un ticket
df_tickets = pd.DataFrame({
    "Cliente": ["Juan Pérez", "Ana Gómez", "Luis Martínez", "Ana Gómez", "Carlos López"],
    "Email": ["juan.perez@email.com", "ana.gomez@email.com", "luis.martinez@email.com", "ana.gomez@email.com", "carlos.lopez@email.com"],
    "idTicket": [1, 2, 3, 4, 5],
    "DetalleTickets": [
        "Problema con la batería del laptop, no carga correctamente. Modelo: Dell XPS 13, Fecha de compra: 2023-01-15, Estado: En garantía.",
        "Pantalla del smartphone presenta líneas verticales. Modelo: iPhone 14 Pro, Fecha de compra: 2023-02-10, Estado: En garantía.",
        "Tablet no enciende después de la actualización. Modelo: Samsung Galaxy Tab S8, Fecha de compra: 2023-03-05, Estado: Fuera de garantía.",
        "Audífonos no se conectan vía Bluetooth. Modelo: Sony WH-1000XM5, Fecha de compra: 2023-04-20, Estado: En garantía.",
        "Monitor tiene píxeles muertos en la esquina superior derecha. Modelo: LG UltraGear 27GP850, Fecha de compra: 2023-05-30, Estado: En garantía."
    ]
})

In [14]:
from llama_index.core.schema import Document

documents_shoppings = [
    Document(
        text=f"Cliente: {record['Cliente']}, Email: {record['Email']}, Detalle: {record['DetalleCompra']}",
        metadata={"Cliente": record["Cliente"], "Email": record["Email"]},
    )
    for record in df_shoppings.to_dict(orient="records")
]

documents_tickets = [
    Document(
        text=f"Ticket ID: {record['idTicket']}, Cliente: {record['Cliente']}, Email: {record['Email']}, Detalle: {record['DetalleTickets']}",
        metadata={"Cliente": record["Cliente"], "Email": record["Email"], "idTicket": record["idTicket"]},
    )
    for record in df_tickets.to_dict(orient="records")
]

index_shoppings = VectorStoreIndex.from_documents(documents_shoppings, show_progress=True)
index_tickets = VectorStoreIndex.from_documents(documents_tickets, show_progress=True)

Generating embeddings: 100%|██████████| 5/5 [00:00<00:00, 23.51it/s]


In [15]:
from llama_index.core.tools import FunctionTool, QueryEngineTool
from llama_index.core.vector_stores import MetadataFilter, MetadataFilters
from google import genai
from google.genai import types
from pydantic import BaseModel, Field

class SentimentSchema(BaseModel):
    """Esquema de análisis del sentimiento"""

    sentimiento: str = Field(..., description="Sentimiento del texto")
    score: str = Field(
        ..., description="Puntuación de sentimiento (0-1). Una puntuación más alta significa un sentimiento más positivo"
    )

def analyze_sentiment(text: str) -> dict:
    """Analizar el sentimiento (positivo, negativo, neutro) de un texto dado."""

    client = genai.Client(api_key=os.environ["GEMINI_API_KEY"])

    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=text,
        config=types.GenerateContentConfig(
            system_instruction="""Eres un experto en análisis de sentimientos. Su tarea consiste en analizar el sentimiento del texto proporcionado.
            Debe seguir el modelo pydantic `SentimentSchema` para devolver la respuesta.""",
            response_mime_type="application/json",
            response_schema=SentimentSchema,
        ),
    )
    return response.text

sentiment_tool = FunctionTool.from_defaults(
    fn=analyze_sentiment,
    name="SentimentAnalyzer",
    description="Analiza emociones en textos de clientes"
)

def insert_ticket(cliente: str, email: str, id_ticket: int, detalle: str) -> str:
    """Inserta un nuevo ticket en el índice de tickets con el detalle del problema reportado.

    Parameters
    ----------
    cliente : str
        Nombre del cliente que reporta el ticket.
    email : str
        Email del cliente que reporta el ticket.
    id_ticket : int
        ID del ticket a insertar.
    detalle : str
        Detalle del problema reportado en el ticket.

    Returns
    -------
    str
        Mensaje de confirmación de inserción.
    """
    try:
        new_ticket = Document(
            text=f"Ticket ID: {id_ticket}, Cliente: {cliente}, Email: {email}, Detalle: {detalle}",
            metadata={"Cliente": cliente, "Email": email, "idTicket": id_ticket},
        )
        index_tickets.insert(new_ticket)
        return "Ticket insertado correctamente."
    except Exception as e:
        return f"Error al insertar el ticket: {str(e)}"

insert_ticket = FunctionTool.from_defaults(
    fn=insert_ticket,
    name="insert_ticket",
    description="Inserta un nuevo ticket en el índice de tickets"
)

def search_ticket_by_id(id_ticket: int) -> str:
    """Busca un ticket específico por su ID exacto
    y devuelve un resumen detallado del problema reportado.

    Parameters
    ----------
    id_ticket : int
        ID del ticket a buscar.

    Returns
    -------
    str
        Resumen detallado del ticket.
    """
    try:
        id_ticket = int(id_ticket)
        filters = MetadataFilters(filters=[MetadataFilter(key="idTicket", value=id_ticket)])
        engine = index_tickets.as_query_engine(
            filters=filters,
            response_mode="tree_summarize",
            llm=llm_gemini,
            system_prompt="Encuentra y resume el contenido del ticket con este ID exacto."
        )
        return engine.query("Describe detalladamente este ticket").response
    except Exception as e:
        return f"No se encontró el ticket con ID {id_ticket}. Error: {str(e)}"

search_ticket_by_id_tool = FunctionTool.from_defaults(
    fn=search_ticket_by_id,
    name="search_ticket_by_id",
    description="Busca información específica de un ticket usando su número ID exacto"
)

def search_ticket_by_email(email_client: str) -> str:
    """Busca un ticket específico por el email del cliente
    y devuelve un resumen detallado del problema reportado.

    Parameters
    ----------
    email_client : str
        Email del cliente a buscar.

    Returns
    -------
    str
        Resumen detallado del ticket.
    """
    try:
        filters = MetadataFilters(filters=[MetadataFilter(key="Email", value=email_client)])
        engine = index_tickets.as_query_engine(
            filters=filters,
            response_mode="tree_summarize",
            llm=llm_gemini,
            system_prompt="Encuentra y resume el contenido del ticket correspondiente al cliente con este email."
        )
        return engine.query("Describe detalladamente este ticket").response
    except Exception as e:
        return f"No se encontró el ticket del cliente con email {email_client}. Error: {str(e)}"

search_ticket_by_email_tool = FunctionTool.from_defaults(
    fn=search_ticket_by_email,
    name="search_ticket_by_email",
    description="Busca información específica de un ticket usando el email del cliente al que pertenece el ticket"
)

def search_shops_by_email(email_client: str) -> str:
    """Busca compras específicas por el email del cliente
    y devuelve un resumen detallado de las compras realizadas.

    Parameters
    ----------
    email_client : str
        Email del cliente a buscar.

    Returns
    -------
    str
        Resumen detallado de las compras.
    """
    try:
        filters = MetadataFilters(filters=[MetadataFilter(key="Email", value=email_client)])
        engine = index_shoppings.as_query_engine(
            filters=filters,
            response_mode="tree_summarize",
            llm=llm_gemini,
            system_prompt="Encuentra y resume el contenido de las compras correspondientes al cliente con este email."
        )
        return engine.query("Describe detalladamente estas compras").response
    except Exception as e:
        return f"No se encontró el ticket del cliente con email {email_client}. Error: {str(e)}"

search_shops_by_email_tool = FunctionTool.from_defaults(
    fn=search_shops_by_email,
    name="search_shops_by_email",
    description="Busca información específica de las compras de un cliente usando su email"
)

In [16]:
from llama_index.tools.tavily_research.base import TavilyToolSpec

tavily_tool = TavilyToolSpec(api_key='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')

In [17]:
response_mode = "refine"

query_engine_shoppings = index_shoppings.as_query_engine(
    response_mode=response_mode,
    verbose=False, 
    system_prompt="Eres un asistente de atención al cliente. Responde preguntas sobre el historial de compras de clientes.", 
    similarity_top_k=2, 
    llm=llm_gemini
)

crm_shoppings_rag_tool = QueryEngineTool.from_defaults(
    query_engine=query_engine_shoppings,
    name="crm_shoppings_rag_tool",
    description="""Herramienta de búsqueda para responder preguntas sobre el historial de compras de clientes. 
    Úsalo cuando necesites información sobre productos comprados anteriormente""",
)

query_engine_tickets = index_tickets.as_query_engine(
    response_mode=response_mode,
    verbose=False, 
    system_prompt="Eres un asistente de atención al cliente. Responde preguntas sobre el historial de tickets clientes.", 
    similarity_top_k=2, 
    llm=llm_gemini,
)

crm_tickets_rag_tool = QueryEngineTool.from_defaults(
    query_engine=query_engine_tickets,
    name="crm_tickets_rag_tool",
    description="""Herramienta de búsqueda para responder preguntas sobre el historial de tickets clientes.
    Úsalo cuando necesites información sobre problemas reportados por clientes""",
)

In [19]:
tools = [sentiment_tool, search_ticket_by_id_tool, search_ticket_by_email_tool, search_shops_by_email_tool, insert_ticket]+tavily_tool.to_tool_list()

from llama_index.core import PromptTemplate

special_instructions = """
Su deber como agente es ayudar a los clientes a resolver problemas relacionados con sus compras y tickets.
Debe responder a las preguntas de los clientes utilizando la información disponible sobre los tickets y las compra,
y en el caso de que se reporte un problema, debes insertar un nuevo ticket en el índice de tickets.

En primer lugar, debe analizar el sentimiento de los mensajes de los clientes.
Si el sentimiento es negativo, debe buscar información sobre las últimas compras del cliente e insertar un nuevo ticket con el detalle del problema reportado.

Si hay una compra de un producto que corresponde al problema, ES IMPORTANTE que hagas una búsqueda en la web, a través de la herramienta `tavily_tool`, 
para encontrar y comparar, en primer lugar el precio de mercado del producto, y en segundo lugar el coste de la reparación.
IMPORTANTE: Utilizar siempre formato JSON válido con comillas dobles para claves y valores.

Por último, en base a la pregunta del cliente, debes redactar una respuesta formal tipo email, incluyendo la información relevante de la compra, 
una posible solución al problema planteado y el presupuesto. Si insertaste un ticket debes confirmar al cliente que se ha creado un ticket para su problema.

En caso de que necesites más información para responder a la pregunta, puedes pedir más detalles al usuario.
"""

# Create the agent with the tools
agent = ReActAgent(
    tools=tools,
    llm=llm_gemini,
    # system_prompt=[...], # Doesn't behave as expected
)

from react_system_prompt import REACT_BASE_PROMPT
# Actualiza el prompt del sistema
formatted_prompt = REACT_BASE_PROMPT.replace(
        "{special_instructions}", special_instructions
)

agent.update_prompts({"react_header": PromptTemplate(formatted_prompt)})

ctx = Context(agent)

In [20]:
query = "Me ha llegado el movil roto. Tiene la pantalla rota y no se encinede. ana.gomez@email.com"

handler = agent.run(query, ctx=ctx)

async for ev in handler.stream_events():
    if isinstance(ev, ToolCallResult):
        print(f"\nCall {ev.tool_name} with {ev.tool_kwargs}\nReturned: {ev.tool_output}")
    if isinstance(ev, AgentStream):
        print(f"{ev.delta}", end="", flush=True)

response = await handler

Thought: The current language of the user is: Spanish. The user is reporting a problem with a broken mobile phone. I need to analyze the sentiment of the message, search for the customer's purchase history, and create a new ticket. I also need to search the web for the market price of the phone and the cost of repair. Finally, I will draft a formal email response.
Action: SentimentAnalyzer
Action Input: {"text": "Me ha llegado el movil roto. Tiene la pantalla rota y no se encinede."}

Call SentimentAnalyzer with {'text': 'Me ha llegado el movil roto. Tiene la pantalla rota y no se encinede.'}
Returned: {
  "sentimiento": "negativo",
  "score": "0.1"
}
Thought: The sentiment is negative. I need to search for the customer's purchase history using their email address, then create a new ticket with the details of the problem. After that, I will search the web for the market price of the phone and the cost of repair.
Action: search_shops_by_email
Action Input: {"email_client": "ana.gomez@em

Parsing nodes: 100%|██████████| 1/1 [00:00<00:00, 816.81it/s]



Call insert_ticket with {'cliente': 'Ana Gómez', 'email': 'ana.gomez@email.com', 'id_ticket': 12345, 'detalle': 'El movil ha llegado roto. Tiene la pantalla rota y no se enciende. El cliente compró un iPhone 14 Pro de 256GB en color negro.'}
Returned: Ticket insertado correctamente.
Thought: A ticket has been created. Now I need to search the web for the market price of the iPhone 14 Pro and the cost of screen repair.
Action: search
Action Input: {"query": "precio iPhone 14 Pro 256GB nuevo", "max_results": 3}
Call search with {'query': 'precio iPhone 14 Pro 256GB nuevo', 'max_results': 3}
Returned: [Document(id_='849ab0e6-8125-4ce5-b56a-1b3c1c75d0e2', embedding=None, metadata={'url': 'https://ar.ebay.com/b/Apple-iPhone-14-Pro-256GB/9355/bn_7118664162'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='En eBay encuentras fabulosas o

In [21]:
print(response)

Estimada Ana Gómez,

Hemos recibido su reporte sobre el problema con su iPhone 14 Pro. Lamentamos mucho los inconvenientes que esto le ha causado.

Según nuestros registros, usted adquirió un iPhone 14 Pro de 256GB en color negro, junto con unos audífonos Sony WH-1000XM5.

Hemos creado un ticket con número 12345 para darle seguimiento a su caso.

El costo de reparación de la pantalla de su iPhone 14 Pro oscila entre los $400 y $500 USD si se realiza directamente con Apple. Sin embargo, existen talleres de reparación externos donde el costo podría ser de aproximadamente 339€.

Le sugerimos las siguientes opciones:

1.  **Reparación con Apple:** Puede contactar directamente al soporte técnico de Apple para solicitar la reparación.
2.  **Reparación en un taller externo:** Le recomendamos buscar talleres de reparación de confianza en su área y solicitar un presupuesto.

Considerando que el precio de un iPhone 14 Pro nuevo es de aproximadamente $475 USD, evaluar el costo de la reparación es

In [22]:
query = "Que tickets tengo? ana.gomez@email.com"

handler = agent.run(query, ctx=ctx)

async for ev in handler.stream_events():
    if isinstance(ev, ToolCallResult):
        print(f"\nCall {ev.tool_name} with {ev.tool_kwargs}\nReturned: {ev.tool_output}")
    if isinstance(ev, AgentStream):
        print(f"{ev.delta}", end="", flush=True)

response = await handler

```tool_code
Thought: The user is asking about the tickets associated with their email address. I should use the search_ticket_by_email tool to find the tickets.
Action: search_ticket_by_email
Action Input: {"email_client": "ana.gomez@email.com"}
```
Call search_ticket_by_email with {'email_client': 'ana.gomez@email.com'}
Returned: The customer, Ana Gómez, with the email address ana.gomez@email.com, has a smartphone screen displaying vertical lines. The device is an iPhone 14 Pro, purchased on 2023-02-10, and is currently under warranty.

Additionally, Ana Gómez, also with the email address ana.gomez@email.com, reported that the mobile arrived broken with a shattered screen and is unable to power on. The customer purchased a black 256GB iPhone 14 Pro.

Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: Estimada Ana Gómez,

Según nuestros registros, usted tiene dos tickets asociados a su correo electrónico ana.gomez@email.com:

1.  Un tick

In [23]:
print(response)

Estimada Ana Gómez,

Según nuestros registros, usted tiene dos tickets asociados a su correo electrónico ana.gomez@email.com:

1.  Un ticket por un problema de líneas verticales en la pantalla de su iPhone 14 Pro, el cual fue comprado el 10 de febrero de 2023 y está bajo garantía.
2.  Un ticket por un iPhone 14 Pro de 256GB color negro que le llegó roto, con la pantalla destrozada y que no enciende.

¿En cuál de estos tickets necesita asistencia?
